# Implémentation

In [5]:
from random import *

In [20]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width, empty=False):
        """
        Constructeur d'un labyrinthe de height cellules de haut et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
        
        if empty == True :
            self.fill()
    
    def info(self):
        """
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = f"{self.height} x {self.width}\n"
        txt += str(self.neighbors)
        return txt

    def __str__(self):
        """
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1, c2):
        """
        Méthode d'instance add_wall
        ajoute un mur entre deux cellules passées en paramètre.
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
            
    def remove_wall(self, c1, c2):
        """
        Méthode d'instance remove_wall
        retire un mur entre deux cellules passées en paramètre.
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Suppression du mur
        if c2 not in self.neighbors[c1]: # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2)   # on l'ajoute
        if c1 not in self.neighbors[c2]: # Si c1 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1)   # on l'ajoute
            
    def get_walls(self):
        """
        Méthode d'instance get_walls
        retourne une liste de liste contenant toute les cellules entre lesquelle il y a un mur.
        """
        wall = []
        visite = []
        for x in range(self.width):
            for y in range(self.height):
                if (x+1,y) not in self.neighbors[x,y] and (x+1<self.height):
                    wall = [(x,y),(x+1,y)]
                    if wall not in visite:
                        visite.append(wall)
                if (x,y+1) not in self.neighbors[x,y] and (y+1<self.height):
                    wall = [(x,y),(x,y+1)]
                    if wall not in visite:
                        visite.append(wall)
        return visite
    
    
    
    def fill(self):
        """
        Méthode d'instance fill
        permet de modifier un labyrinthe pour le remplir de mur
        """
        self.neighbors = {(i,j): set() for i in range(self.height) for j in range (self.width)}

    def empty(self):
        """
        Méthode d'instance empty
        permet de modifier un labyrinthe pour le vider de tout ses murs
        """
        self.fill()
        for i in range(self.height-1):
            for j in range(self.width):
                self.remove_wall((i,j),(i+1,j))
        for i in range(self.height):
            for j in range(self.width-1):
                self.remove_wall((i,j),(i,j+1))


    def get_contiguous_cells(self,c):
        """
        accesseur aux cellules contigues à c
        """
        lst =[(c[0]-1,c[1]),(c[0],c[1]+1),(c[0]+1,c[1]),(c[0],c[1]-1)]
        for x in lst:
            if x[0]<0 or x[1]<0 or x[0]>self.height-1 or x[1]>self.width-1:
                lst.remove(x)
        return lst

    def get_reachable_cells(self,c):
        """
        accesseur aux cellules accessibles depuis c
        """
        return self.neighbors[c]
    
    def get_cells(self):
        """
        Retourne une liste de couple, des coordonnée de chacun des cellules du labyrinthe
        Ne prend rien en paramètre.
        """
        res = []
        for x in range(self.height):
            for y in range(self.width):
                res.append((x,y))
        return res
    
    @classmethod
    def gen_btree(cls,h,w):
        """
        Génère un labyrinthe plein, puis supprimer pour chacune des cellules, de manière aléatoire le mur EST ou SUD
        Prend en paramètre une hauteur {h} et une longueur {w} comme dimension pour le labyrinthe. 
        """
        lab = cls(h,w)
        for cel in lab.get_cells(): # get_cells est une liste comportant toutes les coordonnées de toutes les cellules du labyrinthe
            cE = (cel[0],cel[1]+1) # Attribut les coordonnée de la cellule voisine par le mur EST a une variable
            cS = (cel[0]+1,cel[1]) # Attribut les coordonnée de la cellule voisine par le mur SUD a une variable
            mE = [cel,(cel[0],cel[1]+1)] # Attribut les coordonnées du mur EST de la cellule a une variable
            mS = [cel,(cel[0]+1,cel[1])] # Attribut les coordonnées du mur SUD de la cellule a une variable
            walls = lab.get_walls() 
            if mE in walls or mS in walls:
                if mE in walls and mS in walls:
                    lab.remove_wall(cel,[cE,cS][randint(0,1)])
                elif mE in walls:
                    lab.remove_wall(cel,cE)
                elif mS in walls:
                    lab.remove_wall(cel,cS)
        return lab
    
    def gen_sidewinder(h,w):
        """
        Génère un labyrinthe plein, puis pour chacune des cellules tire aléatoirement si on casse le mur EST ou SUD,
        selon la capacité de chacune des cellule, a pouvoir avoir un voisin qui correspond a des coordonnée se trouvant 
        dans le labyrionthe
        """
        lab = Maze(h, w)
        for i in range(h-1):
            seq = []
            for j in range(w):
                cell = (i,j)
                seq.append(cell)
                vE = None
                if j < w-1: # Vérifie si la cellule peut avoir un voisin avec son mur EST
                    vE = (i,j+1)
                vS = None
                if i < h-1: # Vérifie si la cellule peut avoir un voisin avec son mur SUD
                    vS = (i+1,j)
                ran = randint(0,1) # 0 = Pile | 1 = Face
                if ran == 0 and vE != None:
                    lab.remove_wall(cell,vE) 
                elif ran == 1 and vS != None:
                    ch = choice(seq) # Choisi aléatoirement une cellule dans la liste seq
                    x,y = ch
                    lab.remove_wall(ch,(x+1,y)) 
                    seq = [] 
            seq.append(cell)
            ch = choice(seq)  # Choisi aléatoirement une cellule dans la liste seq
            x,y = ch
            lab.remove_wall(ch,(x+1,y)) 
        for i in range(w-1):
            lab.remove_wall((h-1,i),(h-1,i+1))
        return lab
    
    @classmethod
    def gen_fusion(cls,h,w):
        """
        Génère un labyrinthe plein. On attribut un label de 1 a n dans un dictionnaire. On récupère la liste de tous les murs
        qu'on mélange. Pour chaque mur de la liste extraite on regarde si le label des deux cellule de chaque coté du mur sont differentes.
        Si c'est le cas on casse le mur, on attribut le label d'une des deux cellule a l'autre ainsi qu'a toute les cellule qui porte le
        meme label que la cellule qui vient de recevoir le label de l'autre.
        """
        lab = Maze(h,w,empty=False)
        label = {}
        n=0
        for x in lab.neighbors:
            n+=1
            label[x]=n  
        wall = lab.get_walls()
        shuffle(wall) 
        
        for z in range(len(wall)):
            if label[wall[z][0]]!=label[wall[z][1]]:
                lab.remove_wall(wall[z][0],wall[z][1])
                cell = label[wall[z][1]]
                for y in label:
                    if label[y] == cell:
                        label[y] = label[wall[z][0]]
        
        return lab
    
    @classmethod
    def gen_exploration(cls,h,w):
        lab = cls(h,w,empty=False)
        visite = {}
        for x in lab.neighbors:
            visite[x]="o"
        init = (randint(0,h-1),randint(0,w-1))
        visite[init] = "v"
        pile=[init]
        
        while pile:
            cell = pile.pop()
            ce = lab.get_contiguous_cells(cell)
            shuffle(ce)
            for vois in ce:
                if visite[vois]=="o":
                    lab.remove_wall(cell,vois)
                    visite[vois]="v"
                    pile.append(vois)
        return lab
                    
    def gen_wilson(h,w):
        lab = Maze(h,w)
        cell = lab.get_cells()
        dpt = choice(cell)
        cell.remove(dpt)
        visite = [dpt]
        while len(cell)>0:
            ce = choice(cell)
            snk = [ce]
            while ce in cell:
                nxt = choice(lab.contiguous_cells(ce))
                if nxt in snk:
                    idx = snk.index(nxt)
                    snk = snk[:idx+1]
                else:
                    snk.append(nxt)
                    visite.append(nxt)
                    """Pas finis"""
        return lab
                        
    """Partie 7"""    
    def distance_geo(self,c1,c2):
        """
        Calcule la distance géodésique entre la cellule c1 et c2
        """
        parcour = self.solve_dfs(c1,c2)
        return len(parcour)-1
        
    def distance_man(c1,c2):
        """
        Calcule la distance de Manhattan entre la cellule c1 et c2 c'est-a-dire la diastance qui les séparent si le labyrinthe était vide de tout ses murs.
        """
        return abs(c1[0]-c2[0])+abs(c1[1]-c2[1])
                
        
        
            

In [3]:
laby = Maze(4, 4, True)
print(laby.info())

4 x 4
{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set()}


In [26]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [27]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [28]:
laby.neighbors[(1,3)].remove((2,3))
laby.neighbors[(2,3)].remove((1,3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [29]:
laby.neighbors[(1, 3)].add((2, 3))
laby.neighbors[(2, 3)].add((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [30]:
laby.neighbors[(1, 3)].remove((2, 3))
print(laby)
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

4 x 4
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}


In [31]:
laby.neighbors[(2, 3)].remove((1,3))

In [32]:
laby = Maze(4, 4, empty = True)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [33]:
laby = Maze(4, 4, empty = False)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



# Manipulation de labyrinthes

In [34]:
laby = Maze(5, 5, empty = True)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [35]:
laby.add_wall((0,0), (0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [36]:
laby = Maze(5, 5, empty = True)
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [37]:
laby.remove_wall((0, 0), (0, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [38]:
laby.empty()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [39]:
print(laby.get_walls())

[[(0, 0), (0, 1)], [(0, 1), (1, 1)]]


In [40]:
print(laby.get_contiguous_cells((0,1)))

[(0, 0), (1, 1), (0, 2)]


In [41]:
print(laby.get_reachable_cells((0,1)))

{(0, 2)}


# 5. Génération

## 5.1 Arbre binaire

In [5]:
laby = Maze(4,4)
print(laby.get_cells())

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


In [14]:
laby = Maze.gen_btree(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃           ┃   ┃
┣━━━╋━━━╋   ╋   ┫
┃       ┃   ┃   ┃
┣━━━╋   ╋   ╋   ┫
┃       ┃       ┃
┣━━━╋   ╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



In [72]:
laby = Maze.gen_sidewinder(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃       ┃
┣   ╋   ╋   ╋   ┫
┃   ┃   ┃       ┃
┣   ╋   ╋━━━╋   ┫
┃               ┃
┣   ╋━━━╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



In [73]:
laby = Maze.gen_btree(15, 15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃       ┃       ┃   ┃       ┃       ┃   ┃           ┃   ┃
┣   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃   ┃           ┃       ┃   ┃                   ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃       ┃   ┃           ┃       ┃       ┃       ┃           ┃
┣━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃           ┃   ┃       ┃   ┃           ┃   ┃       ┃       ┃
┣━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃       ┃   ┃       ┃   ┃   ┃       ┃   ┃       ┃           ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃               ┃           ┃       ┃           ┃       ┃   ┃
┣━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃   ┃   ┃       ┃       ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃   ┃       ┃           ┃   ┃   ┃       ┃           ┃       ┃
┣   ╋━━━

In [81]:
laby = Maze.gen_sidewinder(15, 15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃       ┃       ┃   ┃                   ┃   ┃       ┃   ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃   ┃           ┃   ┃   ┃   ┃               ┃       ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃                   ┃       ┃                   ┃           ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃                       ┃   ┃   ┃   ┃               ┃   ┃   ┃
┣━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃           ┃       ┃           ┃   ┃       ┃   ┃   ┃   ┃
┣   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ┫
┃   ┃           ┃       ┃       ┃           ┃       ┃   ┃   ┃
┣   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃                   ┃       ┃   ┃       ┃   ┃   ┃       ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃           ┃       ┃   ┃       ┃       ┃       ┃   ┃   ┃
┣   ╋━━━

In [18]:
laby = Maze.gen_fusion(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃               ┃                   ┃   ┃           ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━┫
┃   ┃       ┃   ┃   ┃       ┃   ┃   ┃       ┃   ┃   ┃       ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━┫
┃   ┃   ┃   ┃           ┃   ┃       ┃                       ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━┫
┃           ┃       ┃       ┃       ┃   ┃   ┃       ┃       ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ┫
┃   ┃           ┃   ┃   ┃               ┃   ┃   ┃       ┃   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━┫
┃               ┃                   ┃   ┃   ┃       ┃       ┃
┣━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━┫
┃           ┃   ┃   ┃       ┃       ┃           ┃           ┃
┣━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃   ┃   ┃       ┃   ┃   ┃       ┃           ┃   ┃       ┃   ┃
┣   ╋   

In [7]:
laby = Maze(15,15)
print(laby.get_contiguous_cells((1,15)))

[(0, 15), (2, 15), (1, 14)]


In [21]:
laby = Maze.gen_exploration(15,15)
print(laby)

KeyError: (15, 14)